# Port Authority TrueTime Example
In this example, we'll collect prediction data for a nearby bus stop, as well as get information about the nearest bus and display it on a simple map

You can checkout the Port Authority's [TrueTime Map](http://truetime.portauthority.org/bustime/map/displaymap.jsp) to visually find the maps and routes you want to work with.

In [1]:
import requests      # the best HTTP request library for python in our humble opinion

# you don't have to use this unless you want to store your API key in a separate file
# we're doing this to keep it secret during the demonstration
# you can comment this out or delete it for your code
from settings import TRUE_TIME_API_KEY     

        
# this is a date and time library to handle the time info
from datetime import datetime

# this library allows us to show a map in a jupyter notebook
import folium

# This stuff is here to pretty print the data we receive from the API
import pprint as PP
pp = PP.PrettyPrinter(indent=2)
pprint = pp.pprint

In [2]:
# We'll need to set up some basic information to make the request

# API Information
API_ROOT_URL = "http://truetime.portauthority.org/bustime/api/v3/"

STOPS_ENDPOINT = "getstops"

PREDICTION_ENDPOINT = "getpredictions"

BUS_ENDPOINT = "getvehicles"

BUS_STOP_ID = 8189   # This is the inbound 71A stop down the street from Code & Supply
                     # You can find the bus stop of your choosing on a map at 
                     # http://truetime.portauthority.org/bustime/map/displaymap.jsp
        
ROUTE_ID = '71A'     # These usually follow the route names you're familiar with. (54c and 54d both fall under 54)
                     # Check the site above for a complete listing.

# When you use this, replace TRUE_TIME_API_KEY with a string of your API key
# e.g. API_KEY = "123k1h4kjh325"
API_KEY = TRUE_TIME_API_KEY

# Set up the parameters that we'll use in every call
BASE_PARAMS = {
    'key': API_KEY, 
    'format': 'json',
    'rtpidatafeed': 'Port Authority Bus'
}   
print(API_KEY)


qNFt32HZTGKjam53VtZbYC3yi


In [3]:
# In version 3 of the TrueTIme API, we need a rtpid for some reason.
# You can see above that we're using `Port Authority Bus`.  Here's how to get it:

r = requests.get(API_ROOT_URL + "getrtpidatafeeds", params=BASE_PARAMS)
pprint((r.json()))

{ 'bustime-response': { 'rtpidatafeeds': [ { 'displayname': 'Light Rail',
                                             'enabled': 'true',
                                             'name': 'Light Rail',
                                             'source': 'BUSTIMEPID',
                                             'visible': 'true'},
                                           { 'displayname': 'Port Authority '
                                                            'Bus',
                                             'enabled': 'true',
                                             'name': 'Port Authority Bus',
                                             'source': 'BUSTIMEPID',
                                             'visible': 'true'}]}}


In [4]:
# Now that everything is set up, let's first look at our bus stop.
stops_url = API_ROOT_URL + STOPS_ENDPOINT
stops_url

'http://truetime.portauthority.org/bustime/api/v3/getstops'

In [5]:
# This will make a dict that has all the key-value pairs in `BASE_PARAMS` plus the `stp_id`
stops_params = {
    **BASE_PARAMS, # the `**` operator breaks apart the dictionary into it's key-value pairs
    'stpid': BUS_STOP_ID
}

stops_params

{'key': 'qNFt32HZTGKjam53VtZbYC3yi',
 'format': 'json',
 'rtpidatafeed': 'Port Authority Bus',
 'stpid': 8189}

In [6]:
# If we hit the `getbusstop` endpoint with our paramters, it should give us info on our stop.
r = requests.get(stops_url, params=stops_params)
bus_stop = r.json()
pprint(bus_stop)

{ 'bustime-response': { 'stops': [ { 'lat': 40.4608079332702,
                                     'lon': -79.93236599999989,
                                     'stpid': '8189',
                                     'stpnm': 'Negley Ave at Friendship Ave'}]}}


In [7]:
# Cool.  That's our bus stop.  There's not much to it, but we can use this information later on in the map!
# Now let's look at some predictions for this bus stop.
prediction_url = API_ROOT_URL + PREDICTION_ENDPOINT
prediction_url

'http://truetime.portauthority.org/bustime/api/v3/getpredictions'

In [8]:
prediction_params = {
    **BASE_PARAMS,
    'stpid': BUS_STOP_ID,
}
prediction_params

{'key': 'qNFt32HZTGKjam53VtZbYC3yi',
 'format': 'json',
 'rtpidatafeed': 'Port Authority Bus',
 'stpid': 8189}

In [9]:
r = requests.get(prediction_url, prediction_params)
pprint(r.json())

{ 'bustime-response': { 'prd': [ { 'des': 'DOWNTOWN',
                                   'dly': False,
                                   'dstp': 5797,
                                   'dyn': 0,
                                   'prdctdn': '6',
                                   'prdtm': '20191018 12:01',
                                   'rt': '71A',
                                   'rtdd': '71A',
                                   'rtdir': 'INBOUND',
                                   'stpid': '8189',
                                   'stpnm': 'Negley Ave at Friendship Ave',
                                   'tablockid': '071A-100',
                                   'tatripid': '9107',
                                   'tmstmp': '20191018 11:54',
                                   'typ': 'A',
                                   'vid': '6256',
                                   'zone': ''},
                                 { 'des': 'DOWNTOWN',
                                

In [10]:
# The above request didn't specifiy a route, so it shows all buses that are arriving within the next 30 mins.
# We can specify just the 71A (pointless, I know since we're looking at the inbound stop, but still...)
prediction_params['rt'] = ROUTE_ID
prediction_params

{'key': 'qNFt32HZTGKjam53VtZbYC3yi',
 'format': 'json',
 'rtpidatafeed': 'Port Authority Bus',
 'stpid': 8189,
 'rt': '71A'}

In [11]:
r = requests.get(prediction_url, params=prediction_params)
prediction_data = r.json()
pprint(prediction_data)

{ 'bustime-response': { 'prd': [ { 'des': 'DOWNTOWN',
                                   'dly': False,
                                   'dstp': 5797,
                                   'dyn': 0,
                                   'prdctdn': '6',
                                   'prdtm': '20191018 12:01',
                                   'rt': '71A',
                                   'rtdd': '71A',
                                   'rtdir': 'INBOUND',
                                   'stpid': '8189',
                                   'stpnm': 'Negley Ave at Friendship Ave',
                                   'tablockid': '071A-100',
                                   'tatripid': '9107',
                                   'tmstmp': '20191018 11:54',
                                   'typ': 'A',
                                   'vid': '6256',
                                   'zone': ''},
                                 { 'des': 'DOWNTOWN',
                                

In [12]:
# Now, if we want to make that human readable, we can put some of that data into a sentence.
# First, we'll filter out just the data we need.  This will be just the closest 71A.
our_prediction = prediction_data['bustime-response']['prd'][0]
our_prediction

{'tmstmp': '20191018 11:54',
 'typ': 'A',
 'stpnm': 'Negley Ave at Friendship Ave',
 'stpid': '8189',
 'vid': '6256',
 'dstp': 5797,
 'rt': '71A',
 'rtdd': '71A',
 'rtdir': 'INBOUND',
 'des': 'DOWNTOWN',
 'prdtm': '20191018 12:01',
 'tablockid': '071A-100',
 'tatripid': '9107',
 'dly': False,
 'dyn': 0,
 'prdctdn': '6',
 'zone': ''}

In [13]:
# Second, we'll find out how far out the bus is.
# We must parse the two datetimes in the data `prdtm` and `tmstmp`
current_time = datetime.strptime(our_prediction['tmstmp'], '%Y%m%d %H:%M')  # the second argument is the format
                                                                            # for more info visit http://strftime.org/
current_time.isoformat()

'2019-10-18T11:54:00'

In [14]:
prediction_time = datetime.strptime(our_prediction['prdtm'], '%Y%m%d %H:%M')
prediction_time.isoformat()

'2019-10-18T12:01:00'

In [15]:
# Let's get a pretty version of that for our message.

pretty_prediction_time = prediction_time.time().isoformat()[0:-3]   

# The `.time().isoformat()` prints a nice format
# And the `[0:-3]` cuts out the last 3 characters (':00'), the seconds,  which aren't useful

pretty_prediction_time

'12:01'

In [16]:
time_left = prediction_time - current_time  # this returns a `timedelta` object 
                                            # https://docs.python.org/3/library/datetime.html#datetime.timedelta
minutes = time_left.seconds / 60
minutes

7.0

In [17]:
# Now that we have the predicted time and how far out that is, we can print it in a nice format!

# Add a little preamble to our message based on how quick it's coming.
warning = "Better hurry up!" if minutes < 10 else "You've got time."

message = "{} The next {} is coming at {} which is in {} minute{}.".format(
    warning, 
    ROUTE_ID,
    pretty_prediction_time,
    int(minutes), # cast it to an int to hide the useless decimal
    '' if minutes == 1 else 's'
)
message

'Better hurry up! The next 71A is coming at 12:01 which is in 7 minutes.'

In [18]:
# Now we can use all this data to show it on a map
# See Eleanor Tutt's Code For Pittsburgh presentation's notebook for more details:
# https://github.com/eleanortutt/codeforpgh-20180613/blob/master/code-for-pgh.ipynb

m = folium.Map(location=[40.4599847,-79.9329759], zoom_start=16)
m

In [19]:
# There's the map, lets add the stop and bus to it.
# we can use the bus_stop data we got earlier
bus_stop

{'bustime-response': {'stops': [{'stpid': '8189',
    'stpnm': 'Negley Ave at Friendship Ave',
    'lat': 40.4608079332702,
    'lon': -79.93236599999989}]}}

In [20]:
# The coord are in the `lat` and `lon` keys.  Let's extract those for easy reuse.
bs = bus_stop['bustime-response']['stops'][0]  # skip past the boilerplate to make our lives easier
bus_stop_coords = [float(bs['lat']), float(bs['lon'])]  # we have to cast them because the API returns strings
bus_stop_coords

[40.4608079332702, -79.93236599999989]

In [21]:
# we can do the same with the bus, we just need it's vehicle id
our_prediction

{'tmstmp': '20191018 11:54',
 'typ': 'A',
 'stpnm': 'Negley Ave at Friendship Ave',
 'stpid': '8189',
 'vid': '6256',
 'dstp': 5797,
 'rt': '71A',
 'rtdd': '71A',
 'rtdir': 'INBOUND',
 'des': 'DOWNTOWN',
 'prdtm': '20191018 12:01',
 'tablockid': '071A-100',
 'tatripid': '9107',
 'dly': False,
 'dyn': 0,
 'prdctdn': '6',
 'zone': ''}

In [22]:
# it's vehicle id is int he 'vid' key
our_prediction['vid']

'6256'

In [23]:
# We can get info on the bus with the bus endpoint
bus_url = API_ROOT_URL + BUS_ENDPOINT
bus_url

'http://truetime.portauthority.org/bustime/api/v3/getvehicles'

In [30]:
# we request the bus we want using the vehicles id, which was in our prediciton response
params = {
    **BASE_PARAMS,
    'vid': our_prediction['vid']
}
params

{'key': 'qNFt32HZTGKjam53VtZbYC3yi',
 'format': 'json',
 'rtpidatafeed': 'Port Authority Bus',
 'vid': '6256'}

In [29]:
# we request that information
r = requests.get(bus_url, params=params)
our_bus = r.json()['bustime-response']['vehicle'][0]  # this time i'm cutting out all the crap from the start
our_bus

{'vid': '6256',
 'rtpidatafeed': 'Port Authority Bus',
 'tmstmp': '20191018 11:56',
 'lat': '40.472220999295594',
 'lon': '-79.92596560618917',
 'hdg': '198',
 'pid': 6561,
 'rt': '71A',
 'des': 'DOWNTOWN',
 'pdist': 4275,
 'dly': False,
 'spd': 15,
 'tatripid': '9107',
 'tablockid': '071A-100',
 'zone': '',
 'mode': 1,
 'psgld': 'N/A'}

In [26]:
bus_coords = [float(our_bus['lat']), float(our_bus['lon'])]
bus_coords

[40.474020410756594, -79.92517565117508]

In [27]:
# now we can make markers
# i used data from the different objects to populat their popups
bus_stop_marker = folium.Marker(bus_stop_coords, 
                                popup=bs['stpnm'])

bus_marker = folium.Marker(bus_coords, 
                           icon=folium.Icon(icon='bus', color='red', prefix='fa'),
                           popup='{} ({})\n{}mins'.format(our_bus['rt'], our_bus['vid'], minutes))

# and add them to the map
bus_stop_marker.add_to(m)
bus_marker.add_to(m)
m